In [ ]:
import numpy as np
import torch

print("Using torch", torch.__version__)


## Creating a tensor

In [ ]:
# Create a tensor with random values between 0 and 1 with the shape [2, 3, 4]
x = torch.rand(2, 3, 4)
print(x)

In [ ]:
# Get shape of tensor
shape = x.shape
print("Shape:", x.shape)

size = x.size()
print("Size:", size)

dim1, dim2, dim3 = x.size()
print("Size:", dim1, dim2, dim3)

In [ ]:
x.device

In [ ]:
x.dtype

## Reshaping a tensor

In [ ]:
W = torch.arange(9) # We can also stack multiple operations in a single line
print("W", W)

In [ ]:
W = W.view(3, 3) # Reshape to 3x3 matrix
print("W", W)

In [ ]:
W = torch.arange(9).view(3, 3) # stack multiple operations in a single line
print("W", W)

## Indexing a Tensor

In [ ]:
print(W[:, 1])   # Second column


In [ ]:
print(W[0])      # First row

In [ ]:
print(W[:2, -1]) # First two rows, last column

In [ ]:
print(W[1:3, :]) # Middle two rows


## Tensor Operations

In [ ]:
# Adding matrices
x1 = torch.rand(2, 3)
x2 = torch.rand(2, 3)
y = x1 + x2

print("X1", x1)
print("X2", x2)
print("Y", y)

In [ ]:
# Matrix Multiplication
x = torch.arange(6)
x = x.view(2, 3)
print("X", x)
print("W", W)

h = torch.matmul(x, W)
print("h", h)

In [ ]:
##

## Training a Neural Network

In [ ]:
import torch.optim as opt
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms

In [ ]:
train_data = torchvision.datasets.GTSRB("/content", split="train", download=True, transform = transforms.Compose([transforms.ToTensor(),
                                                                                               transforms.Resize((224,224))]))
test_data = torchvision.datasets.GTSRB("/content", split="test", download=True, transform = transforms.Compose([transforms.ToTensor(),
                                                                                               transforms.Resize((224,224))]))

In [ ]:
train_data

In [ ]:
import matplotlib.pyplot as plt
figure = plt.figure(figsize=(8, 8))
cols, rows = 5, 5

for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.axis("off")
    plt.imshow(img.swapaxes(1, 2).swapaxes(0, 2)) # C, H, W -> W, H, C
plt.show()

In [ ]:
# Dataloader
train_dl = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_dl = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

In [ ]:
# Model
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model

In [ ]:
# Replace last layer
model.fc = nn.Linear(model.fc.in_features, 43)

In [ ]:
# Set optimizer, loss function, and learning rate
lr = 0.0001
loss_fn = nn.CrossEntropyLoss()
optimizer = opt.SGD(model.parameters(), lr=lr)

In [ ]:
# Check GPU
print("GPU Available: ", torch.cuda.is_available())

In [ ]:
# Set device for training
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Move model to device
model = model.to(device)

In [ ]:
# Training
for i in range(0, 10):
    print(f"Epoch {i+1}\n-------------------------------")

    # Training
    for batch, (X, y) in enumerate(train_dl):
        # 1. Transfer data to device
        X = X.to(device)
        y = y.to(device)

        # 2. Get model prediction
        pred = model(X)

        # 3. Calculate Loss
        loss = loss_fn(pred, y)

        # Zero out gradients for new calculation
        optimizer.zero_grad()

        # 4. Backpropagation
        loss.backward()

        # 5. Update model parameters
        optimizer.step()

        # Print loss
        if batch % 100 == 0:
            loss = loss.item()
            print(f"Training loss: {loss:>7f}")

    size = len(test_dl.dataset)
    num_batches = len(test_dl)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_dl:
            # 1. Transfer data to device
            X = X.to(device)
            y = y.to(device)

            # 2. Get model prediction
            pred = model(X)

            # 3. Evaluate loss and accuracy
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
